In [1]:
print('connect')

connect


In [8]:
import requests

url = 'http://127.0.0.1:8000/image/'

files = {'image': open('media/recommend_image/spao1.jpg', 'rb')}

response = requests.get(url, files=files)

In [44]:
print(response.status_code)
print(response.content)

200
b'"done, class \xed\x8a\xb8\xeb\xa0\x88\xeb\x94\x94\xec\x85\x94\xeb\x84\x90 maybe.."'


In [18]:
# import os
# import requests
#
# url = 'http://127.0.0.1:8000/image/'
# root_path = 'media/recommend_image'
#
# for file_name in os.listdir(root_path)[100:200]:
#     file_path = os.path.join(root_path, file_name)
#     img = open(file_path, 'rb')
#
#     files = {'image': img}
#
#     response = requests.get(url, files=files)

## Initialize Recommaend Meta Database

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import PIL.Image
from ml_sources import executors

In [3]:
device = 'cpu'

executor_segmentation = executors.SegmentationExecutor(device=device)
executor_classification = executors.ClassificationExecutor(device=device)
executor_extract = executors.ExtractExecutor(device=device)

In [9]:
df = pd.read_csv('media/image_meta_data.csv')
df['class'] = np.NaN
df['feature'] = np.NaN
df.head(5)

,쇼핑몰 이름,카테고리,상품명,가격,상품주소,파일 이름,class,feature
0,spao,아우터,크롭 봄버 점퍼_SPJJB49G91,"59,900",http://www.spao.com/product/크롭-봄버-점퍼spjjb49g91...,spao1.jpg,NaN,NaN
1,spao,아우터,틸든 케이블 가디건_SPCKB49G31,"39,900",http://www.spao.com/product/틸든-케이블-가디건spckb49g...,spao2.jpg,NaN,NaN
2,spao,아우터,골지 루즈핏 가디건_SPCKB49W01,"39,900",http://www.spao.com/product/포우먼-골지-루즈핏-가디건spck...,spao3.jpg,NaN,NaN
3,spao,아우터,소프트 하찌 오버핏 베스트_SPKVB49W02,"35,900",http://www.spao.com/product/포우먼-소프트-하찌-오버핏-베스트...,spao4.jpg,NaN,NaN
4,spao,아우터,브이넥 스웨터 베스트_SPKVB49W01,"29,900",http://www.spao.com/product/포우먼-브이넥-스웨터-베스트spk...,spao5.jpg,NaN,NaN


In [10]:
def img_analysis(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    segmentationed_img: PIL.Image.Image = executor_segmentation.execute(image=img)

    classification_label: str = executor_classification.execute(image=segmentationed_img)
    fv = executor_extract.execute(image=segmentationed_img)

    return classification_label, fv

In [39]:
root_path = os.path.join('media', 'recommend_image')

for each in df.head(3).iterrows():
    idx = each[0]
    file_name = each[1]['파일 이름']
    file_path = os.path.join(root_path, file_name)

    label, fv = img_analysis(file_path)
    # df.iloc[idx, 6] = label
    df.iloc[idx, 7] = fv.tobytes()

    print(idx, file_path)
    print(label, fv)
    print('='*10)

1/1 [==============================] - 0s 141ms/step
0 media\recommend_image\spao1.jpg
오피스룩 [0.         0.01794693 0.03533994 ... 0.         0.         0.02775439]
1/1 [==============================] - 0s 110ms/step
1 media\recommend_image\spao2.jpg
트레디셔널 [0. 0. 0. ... 0. 0. 0.]
1/1 [==============================] - 0s 110ms/step
2 media\recommend_image\spao3.jpg
트레디셔널 [0.         0.01550928 0.         ... 0.         0.         0.05407573]


In [46]:
np.frombuffer(df.head(3)['feature'][2], dtype=np.float32)

array([0.        , 0.01550928, 0.        , ..., 0.        , 0.        ,
       0.05407573], dtype=float32)